<a href="https://colab.research.google.com/github/mattshu0410/medical-reasoning-interp/blob/main/notebooks/BMJ_AI_CoT_Exploration.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Loading Dataset & Libraries

In [ ]:
!pip install git+https://github.com/vgel/repeng.git

  Cloning https://github.com/vgel/repeng.git to /tmp/pip-req-build-5f4ilqj_
  Running command git clone --filter=blob:none --quiet https://github.com/vgel/repeng.git /tmp/pip-req-build-5f4ilqj_
  Resolved https://github.com/vgel/repeng.git to commit 0ba7196d81f7c5de28b1159c9d1732440afd97ce
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 96.2/96.2 kB 5.5 MB/s eta 0:00:00
  Created wheel for repeng: filename=repeng-0.5.0-py3-none-any.whl size=14698 sha256=0ba8935ef87f05f8c8fc4894e6468b9091bae04dc5314cbbf804d605f6c08a3e
  Stored in directory: /tmp/pip-ephem-wheel-cache-12qhriav/wheels/71/1f/2b/99e795b09b9d02c27888ca18ffe730feb7aa7d6ec31ee292c0
Successfully built repeng


In [ ]:
!pip install pydantic-ai
!pip install "pydantic-ai-slim[bedrock]"
!pip install "pydantic-ai-slim[anthropic]"

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 92.5/92.5 kB 5.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 347.6/347.6 kB 17.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 55.1/55.1 kB 5.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.9/12.9 MB 61.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 337.3/337.3 kB 6.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.7/43.7 kB 3.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 139.3/139.3 kB 12.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 295.4/295.4 kB 25.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 48.4/48.4 kB 4.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 144.4/144.4 kB 8.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 135.4/135.4 kB 10.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 228.2/228.2 kB 21.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [ ]:
import asyncio
import pandas as pd
from datasets import load_dataset
import re
from pydantic import BaseModel, Field
from pydantic_ai import Agent, PromptedOutput
from pydantic_ai.models.bedrock import BedrockConverseModel, BedrockModelSettings
from pydantic_ai.models.anthropic import AnthropicModel, AnthropicModelSettings
from pydantic_ai.providers.anthropic import AnthropicProvider
from pydantic_ai.models.groq import GroqModel, GroqModelSettings
from pydantic_ai.providers.groq import GroqProvider
from pydantic_ai.settings import ModelSettings
from typing import List, Literal, Optional
from tqdm.asyncio import tqdm
import numpy as np

In [ ]:
# ds = load_dataset("zou-lab/MedCaseReasoning", "default")
# Filtered Dataset without case prompts with diagnosis in them
ds = load_dataset("tmknguyen/MedCaseReasoning-filtered", "default")

data/train-00000-of-00001.parquet:   0%|          | 0.00/4.58M [00:00<?, ?B/s]

data/test-00000-of-00001.parquet:   0%|          | 0.00/2.09M [00:00<?, ?B/s]

data/val-00000-of-00001.parquet:   0%|          | 0.00/183k [00:00<?, ?B/s]

Generating train split:   0%|          | 0/607 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/252 [00:00<?, ? examples/s]

Generating val split:   0%|          | 0/22 [00:00<?, ? examples/s]

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


Output from Thao's DeepSeek

In [ ]:
# Replace 'path/to/your/file.csv' with the actual path to your file in Google Drive
try:
  df = pd.read_csv('/content/drive/MyDrive/Research/BMJ AI Paper/MedCaseReasoning DeepSeek 100 traces 100 cases.csv')
  display(df.head())
except FileNotFoundError:
  print("Error: File not found. Please check the path to your file.")

,pmcid,repeat_index,case_prompt,true_diagnosis,predicted_diagnosis,is_correct,reasoning_trace,verification_response
0,PMC4722580,95,A 19-year-old male presented with pain in the ...,cleidocranial dysplasia,Cleidocranial dysplasia,True,"First, I need to read the case presentation ca...",y
1,PMC4722580,91,A 19-year-old male presented with pain in the ...,cleidocranial dysplasia,Cleidocranial Dysplasia,True,"First, I need to read the case presentation ca...",y
2,PMC4722580,7,A 19-year-old male presented with pain in the ...,cleidocranial dysplasia,Cleidocranial dysplasia,True,"First, I need to read the case presentation ca...",y
3,PMC4722580,26,A 19-year-old male presented with pain in the ...,cleidocranial dysplasia,Cleidocranial dysplasia,True,"First, I need to read the case presentation ca...",y
4,PMC4722580,32,A 19-year-old male presented with pain in the ...,cleidocranial dysplasia,cleidocranial dysplasia,True,"First, I need to read the case presentation ca...",y


In [ ]:
ds["train"].column_names
medcase_df = ds["train"].to_pandas()
medcase_df

,pmcid,title,journal,article_link,publication_date,text,case_prompt,diagnostic_reasoning,final_diagnosis
0,PMC10995541,Bell’s palsy after concomitant chemoradiothera...,Journal of Dental Anesthesia and Pain Medicine,https://www.ncbi.nlm.nih.gov/pmc/articles/PMC1...,2024-03-28,INTRODUCTION Bell's palsy is an unexplained ep...,A 48-year-old man with squamous cell carcinoma...,1. Tumor progression was considered but exclud...,Bell’s palsy
1,PMC4387368,Mycoplasmal Upper Respiratory Infection Presen...,Infectious Disease Reports,https://www.ncbi.nlm.nih.gov/pmc/articles/PMC4...,2015-02-24,Introduction Mycoplasma is a virulent organism...,A previously healthy 24-year-old Hispanic man ...,1. Drug-induced bullous pemphigoid — “Pertinen...,Leukocytoclastic vasculitis
2,PMC9366264,The IRIS paradox: Imaging findings in a case o...,Respirology Case Reports,https://www.ncbi.nlm.nih.gov/pmc/articles/PMC9...,2022-08-11,INTRODUCTION Immune reconstitution inflammator...,A 57‐year‐old woman with a history of HIV infe...,1. Cytomegalovirus pneumonia considered — “Cyt...,Pneumocystis-IRIS
3,PMC8938864,Gradenigo's syndrome presenting as IX and X cr...,eNeurologicalSci,https://www.ncbi.nlm.nih.gov/pmc/articles/PMC8...,2022-03-17,1 Introduction Gradenigo's syndrome is a const...,A 55-year-old man presented with a 3-month his...,1. Space-occupying lesion near the jugular for...,Gradenigo syndrome
4,PMC9788876,Psoriasiform Dermatitis in a COVID-19 Patient,Case Reports in Dermatological Medicine,https://www.ncbi.nlm.nih.gov/pmc/articles/PMC9...,2022-12-16,1. Introduction Psoriasis is a chronic inflamm...,A 52‐year‐old man with no significant medical ...,1. Drug‐induced eruption excluded — “There was...,psoriasis
...,...,...,...,...,...,...,...,...,...
602,PMC3546017,Static winging of the scapula caused by osteoc...,Journal of Medical Case Reports,https://www.ncbi.nlm.nih.gov/pmc/articles/PMC3...,2012-10-25,Introduction Winging of the scapula (scapula a...,A 35-year-old Caucasian woman presented with s...,1. Serratus anterior palsy was considered for ...,Osteochondroma
603,PMC10871805,A case of tumor-like inflammatory demyelinatin...,São Paulo Medical Journal,https://www.ncbi.nlm.nih.gov/pmc/articles/PMC1...,2014-11-07,INTRODUCTION Tumor-like inflammatory demyelina...,A 24-year-old right-handed man presented with ...,1. Infectious causes ruled out — “Gram stainin...,Tumor-like inflammatory demyelinating disease
604,PMC11867736,Long-Term Survival after Treatment with Induct...,Surgical Case Reports,https://www.ncbi.nlm.nih.gov/pmc/articles/PMC1...,2025-02-22,Abbreviations CUP carcinoma of unknown primary...,A 46-year-old man with a 90 pack-year smoking ...,1. Primary lung adenocarcinoma was considered ...,Carcinoma of unknown primary
605,PMC8457908,Transient visual field loss after COVID-19 vac...,American Journal of Ophthalmology Case Reports,https://www.ncbi.nlm.nih.gov/pmc/articles/PMC8...,2021-09-23,1 Introduction Since the global spreading of t...,A 42-year-old Thai male ophthalmologist with w...,1. Acute cerebral infarction was considered bu...,acute arterial vasospasm


# Get Reasoning Trace

## Generic Helper Functions

In [ ]:
class VerificationOutput(BaseModel):
  """
  Structured output for diagnosis verification.
  """
  is_correct: Literal["y", "n"] = Field(
      description="Whether the predicted diagnosis matches the true diagnosis (y or n)."
  )
  explanation: str = Field(
      default="",
      description="Brief explanation of the verification decision."
  )

In [ ]:
class SentenceClassification(BaseModel):
    """Classification for a single sentence"""
    sentence: str = Field(description="The original sentence from the reasoning trace")
    classification: Literal[
        "process_control",
        "evidence_gathering",
        "hypothesis_generation",
        "hypothesis_testing",
        "diagnosis"
    ] = Field(description="The behavioral category of this sentence")

class ReasoningClassificationOutput(BaseModel):
    """Structured output for reasoning trace classification"""
    classifications: List[SentenceClassification] = Field(
        description="List of sentences with their classifications"
    )

In [ ]:
def create_diagnosis_agent(
    model,
    ) -> Agent:
    """
    Create an agent for medical diagnosis using native text output.

    Args:
        model: Configured Bedrock model
    """

    instructions = """
    You are a medical expert. Read case presentations and provide diagnoses.
    Follow this exact output format:
    <answer>
    ...the name of the disease/entity...
    </answer>
    """
    agent = Agent(
        model,
        output_type=str,
        instructions=instructions,
    )
    return agent

def create_verification_agent(
    model,
) -> Agent:
    """
    Create an agent for verifying diagnosis correctness.
    Uses structured output since verification doesn't need chain of thought.

    Args:
        model: Configured Bedrock model
    """
    instructions = """
    You are evaluating whether a predicted diagnosis matches the true diagnosis.
    Consider semantic equivalence - different phrasings of the same condition should be considered correct.
    For example:
    - "Myocardial infarction" and "Heart attack" are the same
    - "Type 2 diabetes mellitus" and "Diabetes type 2" are the same
    - "Acute appendicitis" and "Appendicitis" are the same
    Be strict but fair in your evaluation.
    """

    agent = Agent(
        model,
        output_type=PromptedOutput(
            VerificationOutput,
            description="Diagnosis verification result"
        ),
        instructions=instructions,
    )
    return agent

In [ ]:
async def run_single_diagnosis(
    agent: Agent,
    case_prompt: str,
    pmcid: str,
    repetition_num: int
  ) -> dict:
    """
    Run a single diagnosis attempt.

    Args:
        agent: The diagnosis agent
        case_prompt: Prompt for the case
        pmcid: PMCID of the case
        repetition_num: Repetition number

    Returns:
        Dictionary with result data
    """
    # Format the prompt with the case
    prompt = f"""
    Read the following case presentation and give the most likely diagnosis.
    You must think very carefully, considering the information available to you.
    You must consider a very broad range of differentials.
    You must take care not to prematurely anchor on a diagnosis or overlook critical negative findings.
    Then, output the final diagnosis (just the name of the disease/entity) within the tags <answer> ... </answer>.
    ---------------------------------------
    CASE PRESENTATION
    ---------------------------------------
    {case_prompt}
    ---------------------------------------
    OUTPUT TEMPLATE
    ---------------------------------------
    <answer>
    ...the name of the disease/entity...
    </answer>"""

    try:
      result = await agent.run(prompt)
      model_response = result.all_messages()[-1]
      reasoning = model_response.thinking if hasattr(model_response, 'thinking') else None
      text_output = model_response.text if hasattr(model_response, 'text') else None
      if text_output:
        answer_match = re.search(r'<answer>(.*?)</answer>', text_output, re.DOTALL | re.IGNORECASE)
        diagnosis = answer_match.group(1).strip() if answer_match else None
      else:
        diagnosis = None

      return {
          'pmcid': pmcid,
          'repetition': repetition_num,
          'reasoning_trace': reasoning,
          'predicted_diagnosis': diagnosis,
          'raw_response': model_response,
          'parsing_successful': reasoning is not None and diagnosis is not None,
          'success': True,
          'error': None
      }
    except Exception as e:
      return {
          'pmcid': pmcid,
          'repetition': repetition_num,
          'reasoning_trace': None,
          'predicted_diagnosis': None,
          'raw_response': None,
          'parsing_successful': False,
          'success': False,
          'error': str(e)
      }


async def run_case_with_repetitions(
    agent: Agent,
    row: pd.Series,
    n_repetitions: int = 100,
    max_concurrent: int = 10
) -> List[dict]:
    """
    Run a single case n times with concurrency control.

    Args:
        agent: The diagnosis agent
        row: DataFrame row with case data
        n_repetitions: Number of times to repeat (default 100)
        max_concurrent: Maximum concurrent API calls

    Returns:
        List of result dictionaries
    """
    semaphore = asyncio.Semaphore(max_concurrent)

    async def run_with_semaphore(rep_num):
        async with semaphore:
            return await run_single_diagnosis(
                agent,
                row['case_prompt'],
                row['pmcid'],
                rep_num
            )

    tasks = [run_with_semaphore(i) for i in range(n_repetitions)]
    results = await asyncio.gather(*tasks)

    # Add true diagnosis to each result
    for result in results:
        result['true_diagnosis'] = row['final_diagnosis']
        result['case_prompt'] = row['case_prompt']

    return results

In [ ]:
async def verify_diagnosis(
    agent: Agent,
    predicted_diagnosis: str,
    true_diagnosis: str
) -> dict:
    """
    Verify if a predicted diagnosis matches the true diagnosis.

    Returns:
        Dictionary with verification results
    """
    prompt = f"""Is the predicted diagnosis correct?

    Predicted diagnosis: {predicted_diagnosis}
    True diagnosis: {true_diagnosis}

    Answer y (correct) or n (incorrect)."""

    try:
        result = await agent.run(prompt)
        output = result.output

        return {
            'is_correct': output.is_correct,
            'explanation': output.explanation,
            'success': True,
            'error': None
        }
    except Exception as e:
        return {
            'is_correct': None,
            'explanation': None,
            'success': False,
            'error': str(e)
        }


async def verify_results(
    agent: Agent,
    results_df: pd.DataFrame,
    max_concurrent: int = 20
) -> pd.DataFrame:
    """
    Verify all diagnosis results.

    Args:
        verification_agent: The verification agent
        results_df: DataFrame with diagnosis results
        max_concurrent: Maximum concurrent verification calls

    Returns:
        DataFrame with verification results added
    """
    semaphore = asyncio.Semaphore(max_concurrent)

    async def verify_with_semaphore(row):
        async with semaphore:
            if pd.isna(row['predicted_diagnosis']) or row['predicted_diagnosis'] is None:
                return None
            return await verify_diagnosis(
                agent,
                row['predicted_diagnosis'],
                row['true_diagnosis']
            )

    print("Running verification...")
    tasks = []
    for idx, row in results_df.iterrows():
        tasks.append(verify_with_semaphore(row))

    verification_results = []
    for task in tqdm(asyncio.as_completed(tasks), total=len(tasks), desc="Verifying"):
        result = await task
        verification_results.append(result)

    # Add verification columns
    results_df['is_correct'] = [
        r['is_correct'] if r and r['success'] else None
        for r in verification_results
    ]
    results_df['verification_explanation'] = [
        r['explanation'] if r and r['success'] else None
        for r in verification_results
    ]

    return results_df


In [ ]:
def create_classification_agent(model) -> Agent:
    """
    Create an agent for classifying reasoning trace sentences.

    Args:
        model: Configured model for classification
    """
    instructions = """
    You are an expert at analyzing medical reasoning patterns.

    Your task is to classify each sentence in a reasoning trace according to this taxonomy:

    1. **Process control**: Non-medical sentences that plan the overall problem-solving trajectory.
       Examples: "Let me analyze this case carefully", "Okay, let me tackle this case"

    2. **Evidence gathering**: Restating facts from the case without interpretation.
       Examples: "45-year-old woman with heavy bleeding", "CT showed bilateral nodules"

    3. **Hypothesis generation**: Generating differential diagnoses.
       Examples: "This could be benign metastasizing leiomyoma", "Possible diagnoses include...",
       "Wait, what about leiomyosarcoma?", "Let me reconsider - could this be BML?"

    4. **Hypothesis testing**: Combining medical facts with case details to weigh differentials.
       Examples: "Miliary TB can cause small nodules, but they're usually more diffuse",
       "The pathology showing smooth muscle without malignancy argues against leiomyosarcoma"

    5. **Diagnosis**: Locking in on a final diagnosis.
       Examples: "Therefore, this is benign metastasizing leiomyoma",
       "The diagnosis is BML", "So putting it together: benign metastasizing leiomyoma"

    Important notes:
    - Use surrounding context to make better judgments
    - Models may backtrack (generate new hypotheses after reaching diagnosis)
    - Some sentences may be ambiguous - use your best judgment
    - Classify EVERY sentence, even short ones
    """

    agent = Agent(
        model,
        output_type=PromptedOutput(
            ReasoningClassificationOutput,
            description="Sentence-by-sentence classification of reasoning trace"
        ),
        instructions=instructions,
    )
    return agent

async def classify_reasoning_trace(
    agent: Agent,
    reasoning_trace: str,
    pmcid: str
) -> dict:
    """
    Classify sentences in a reasoning trace.

    Args:
        agent: Classification agent
        reasoning_trace: The reasoning text to classify
        pmcid: Case identifier

    Returns:
        Dictionary with classification results
    """
    # Split into sentences (simple approach - you can use nltk for better results)
    import re
    sentences = re.split(r'(?<=[.!?])\s+', reasoning_trace)
    sentences = [s.strip() for s in sentences if s.strip()]

    prompt = f"""
    Classify each sentence in the following medical reasoning trace.

    REASONING TRACE:
    {reasoning_trace}

    Please classify every sentence according to the taxonomy provided in your instructions.
    """

    try:
        result = await agent.run(prompt)
        output = result.output

        return {
            'pmcid': pmcid,
            'classifications': output.classifications,
            'num_sentences': len(output.classifications),
            'success': True,
            'error': None
        }
    except Exception as e:
        return {
            'pmcid': pmcid,
            'classifications': None,
            'num_sentences': 0,
            'success': False,
            'error': str(e)
        }

async def classify_all_traces(
    agent: Agent,
    results_df: pd.DataFrame,
    max_concurrent: int = 10
) -> pd.DataFrame:
    """
    Classify reasoning traces for all results.

    Args:
        agent: Classification agent
        results_df: DataFrame with reasoning_trace column
        max_concurrent: Maximum concurrent API calls

    Returns:
        DataFrame with sentence-level classifications
    """
    semaphore = asyncio.Semaphore(max_concurrent)

    async def classify_with_semaphore(row):
        async with semaphore:
            if pd.isna(row['reasoning_trace']) or not row['reasoning_trace']:
                return None
            return await classify_reasoning_trace(
                agent,
                row['reasoning_trace'],
                row['pmcid']
            )

    print("Classifying reasoning traces...")
    tasks = []
    for idx, row in results_df.iterrows():
        tasks.append(classify_with_semaphore(row))

    classification_results = []
    for task in tqdm(asyncio.as_completed(tasks), total=len(tasks), desc="Classifying"):
        result = await task
        classification_results.append(result)

    # Flatten into sentence-level dataframe
    rows = []
    for result in classification_results:
        if result and result['success'] and result['classifications']:
            for classification in result['classifications']:
                rows.append({
                    'pmcid': result['pmcid'],
                    'sentence': classification.sentence,
                    'classification': classification.classification
                })

    return pd.DataFrame(rows)

## Set Preferences

You need to set your API keys in environment variables in Colab. I had to write explicitly because implicit wasn't working. Don't judge.

In [ ]:
from google.colab import userdata
ANTHROPIC_API_KEY = userdata.get('ANTHROPIC_API_KEY')
GROQ_API_KEY = userdata.get('GROQ_API_KEY')

# Number of repetitions per case (default 100)
n_repetitions: int = 1
# Maximum concurrent API calls (if you get errors regarding rate limits drop this)
max_concurrent: int = 1
output_path: str = "results_df.parquet"


### Bedrock

Warning this is a pain in the arse to get set up. Use as a last resort.
https://docs.aws.amazon.com/bedrock/latest/APIReference/API_runtime_Converse.html#API_runtime_Converse_RequestSyntax
https://docs.aws.amazon.com/bedrock/latest/userguide/kb-test-configure-reasoning.html

In [ ]:

# Model Settings

# diagnosis_model = BedrockConverseModel(
#     model_name="us.anthropic.claude-sonnet-4-20250514-v1:0",
#     settings=BedrockModelSettings(
#         bedrock_additional_model_requests_fields={
#             "inferenceConfig": {
#                 "temperature": 0.8,
#                 "topP": 0.95
#                 },
#             'thinking': {
#               'type': 'enabled',
#               'budget_tokens': 2048,
#               },
#             }
#     )
# )

# verification_model = BedrockConverseModel(
#     model_name="us.anthropic.claude-sonnet-4-20250514-v1:0",
#     settings=BedrockModelSettings(
#         bedrock_additional_model_requests_fields={
#             "inferenceConfig": {
#                 "temperature": 0.8,
#                 "topP": 0.95
#                 },
#         }
#     )
# )




### Claude or Opus

In [ ]:
# Note that you can't set temperature to anything other than 1 with thinking enabled
diagnosis_model = AnthropicModel(
    model_name = 'claude-sonnet-4-0', # Can swap out for claude-3-5-sonnet-latest
    settings = AnthropicModelSettings(
        anthropic_thinking={'type': 'enabled', 'budget_tokens': 1024},
    ),
    provider = AnthropicProvider(api_key = ANTHROPIC_API_KEY)
)

verification_model = AnthropicModel(
    model_name = 'claude-3-5-sonnet-latest',
    settings = AnthropicModelSettings(
        temperature = 0.8,
        top_p = 0.95
    ),
    provider = AnthropicProvider(api_key = ANTHROPIC_API_KEY)
)

### Qwen3 & GPT OSS


In [ ]:
diagnosis_model = GroqModel(
    model_name = 'qwen/qwen3-32b', # Can swap out with openai/gpt-oss-120b
    settings = GroqModelSettings(
        groq_reasoning_format='parsed',
    ),
    provider=GroqProvider(api_key=GROQ_API_KEY)
)

# We can still use Claude just for verification
verification_model = AnthropicModel(
    model_name = 'claude-3-5-sonnet-latest',
    settings = AnthropicModelSettings(
        temperature = 0.8,
        top_p = 0.95
    ),
    provider = AnthropicProvider(api_key = ANTHROPIC_API_KEY)
)

## Run Model

In [ ]:

# Create agents
diagnosis_agent = create_diagnosis_agent(diagnosis_model)
verification_agent = create_verification_agent(verification_model)
classification_agent = create_classification_agent(verification_model)

In [ ]:
medcase_df

,Unnamed: 0,pmcid,title,journal,article_link,publication_date,text,case_prompt,diagnostic_reasoning,final_diagnosis
0,0,PMC4863315,A diffuse traumatic neuroma in the palate: a c...,Journal of Medical Case Reports,https://www.ncbi.nlm.nih.gov/pmc/articles/PMC4...,2016-05-11,Background A traumatic neuroma is a hyperplast...,A 30-year-old man presented with several month...,1. Infection or inflammation was considered gi...,Traumatic neuroma
1,1,PMC5728002,Anterior Elbow Pain Caused by Compression of t...,Journal of Orthopaedic Case Reports,https://www.ncbi.nlm.nih.gov/pmc/articles/PMC5...,2017,Learning Points for this Article: Compression ...,A 27-year-old left-hand–dominant man presented...,1. Biceps tendinopathy — “provocative testing ...,Lateral antebrachial cutaneous nerve compression
2,3,PMC4217568,Endosalpingiosis in Postmenopausal Elderly Women,Journal of Menopausal Medicine,https://www.ncbi.nlm.nih.gov/pmc/articles/PMC4...,2014-04-28,Introduction In gynecology endosalpingiosis is...,"A 75-year-old woman, gravida 2, postmenopausal...",1. Mucinous cystadenoma was the preoperative c...,Endosalpingiosis
3,4,PMC11003597,An Uncommon Complication of a Common Tropical ...,Indian Journal of Nephrology,https://www.ncbi.nlm.nih.gov/pmc/articles/PMC1...,2024-03-30,"Introduction Hemophagocytic syndrome, also cal...","A 27-year-old man, 6 months after deceased-don...",1. Immunosuppressant toxicity — “The most comm...,Hemophagocytic lymphohistiocytosis
4,5,PMC7042146,What is the cause of hypotension? A rare compl...,European Heart Journal: Case Reports,https://www.ncbi.nlm.nih.gov/pmc/articles/PMC7...,2019-10-22,Learning points Differential diagnosis of hypo...,A 75-year-old woman with treated hypertension ...,1. Pericardial tamponade excluded — “This show...,Intramural haematoma
...,...,...,...,...,...,...,...,...,...,...
13087,14484,PMC6848952,Relapsing polychondritis with large airway inv...,Respirology Case Reports,https://www.ncbi.nlm.nih.gov/pmc/articles/PMC6...,2019-11-12,Introduction Relapsing polychondritis is a rar...,A 57-year-old woman with a 20-year history of ...,1. Tracheobronchomalacia from relapsing polych...,relapsing polychondritis
13088,14485,PMC10276980,Idiopathic pneumoperitoneum in a single case: ...,Journal of Surgical Case Reports,https://www.ncbi.nlm.nih.gov/pmc/articles/PMC1...,2023-06-17,INTRODUCTION Pneumoperitoneum is known to affe...,A 34-year-old man with type II diabetes mellit...,1. Gastric or duodenal ulcer perforation was c...,Idiopathic pneumoperitoneum
13089,14486,PMC2518156,Challenges in the prenatal and post-natal diag...,Journal of Medical Case Reports,https://www.ncbi.nlm.nih.gov/pmc/articles/PMC2...,2008-08-01,Introduction Cystic hygromas are slow-growing ...,A 27-year-old primigravida at 38 weeks’ gestat...,1. Congenital diaphragmatic hernia was conside...,cystic hygroma
13090,14487,PMC7641677,Marchiafava–Bignami Disease Associated with Sp...,Case Reports in Neurological Medicine,https://www.ncbi.nlm.nih.gov/pmc/articles/PMC7...,2020-10-28,1. Introduction Marchiafava–Bignami Disease (M...,A 53-year-old man was admitted with a two‐mont...,1. Aggressive multiple sclerosis was considere...,Marchiafava–Bignami disease


In [ ]:
test_df = medcase_df.loc[medcase_df["pmcid"] == "PMC8651751"]
test_df = medcase_df[1:200]

In [ ]:
# Run diagnosis for all cases
all_results = []


print(f"Running diagnosis for {len(test_df)} cases with {n_repetitions} repetitions each...")
for idx, row in tqdm(test_df.iterrows(), total=len(test_df), desc="Cases"):
    case_results = await run_case_with_repetitions(
        diagnosis_agent,
        row,
        n_repetitions,
        max_concurrent
    )
    all_results.extend(case_results)

# Create results DataFrame
results_df = pd.DataFrame(all_results)

# Run verification
results_df = await verify_results(
    verification_agent,
    results_df,
    max_concurrent=max_concurrent * 2  # Verification is faster
)



Running diagnosis for 199 cases with 1 repetitions each...


Cases: 100%|██████████| 199/199 [13:23<00:00,  4.04s/it]


Running verification...


Verifying: 100%|██████████| 199/199 [02:58<00:00,  1.11it/s]


In [ ]:
output_dir = "/content/drive/MyDrive/Colab Notebooks/BMJ_AI"
results_df.to_csv(f"{output_dir}/reasoning_traces.csv", index=False)

In [ ]:
results_df = results_df.merge(
    medcase_df[['pmcid', 'diagnostic_reasoning']],
    on='pmcid',
    how='left'
)

In [ ]:
results_df = results_df[1:30]

In [ ]:
from IPython.display import display, HTML

def show_case(idx):
    row = results_df.iloc[idx]
    html = f"""
    <div style='border: 2px solid #4CAF50; padding: 15px; margin: 10px; border-radius: 5px;'>
        <h3 style='color: #4CAF50;'>Case {idx}: {row['pmcid']}</h3>
        <p><strong>True Diagnosis:</strong> <span style='color: green;'>{row['true_diagnosis']}</span></p>
        <p><strong>Predicted:</strong> <span style='color: {"green" if row["is_correct"] else "red"};'>{row['predicted_diagnosis']}</span></p>
        <p><strong>Correct:</strong> {row['is_correct']}</p>
        <hr>
        <h4>Case Prompt:</h4>
        <div style='padding: 10px; border-radius: 5px; max-height: 400px; overflow-y: auto;'>
            {row['case_prompt'].replace('\n', '<br>')}
        </div>
        <h4>Model Reasoning Trace:</h4>
        <div style='padding: 10px; border-radius: 5px; max-height: 400px; overflow-y: auto;'>
            {row['reasoning_trace'].replace('\n', '<br>')}
        </div>
        <h4>Dataset Reasoning Trace:</h4>
        <div style='padding: 10px; border-radius: 5px; max-height: 400px; overflow-y: auto;'>
            {row['diagnostic_reasoning'].replace('\n', '<br>')}
        </div>
        <h4>Verification Explanation:</h4>
        <div style='padding: 10px; border-radius: 5px; max-height: 400px; overflow-y: auto;'>
            {row['verification_explanation'].replace('\n', '<br>')}
        </div>
    </div>
    """
    display(HTML(html))

from ipywidgets import interact, Dropdown

idx = results_df.index.tolist()

@interact(case_idx=Dropdown(options=idx, description='Case:'))
def explore_incorrect(case_idx):
    show_case(case_idx)

In [ ]:
sentence_df = await classify_all_traces(
        classification_agent,
        results_df,
        max_concurrent=max_concurrent
    )

sentence_df.to_csv(f"{output_dir}/sentence_classifications.csv", index=False)
print(sentence_df['classification'].value_counts())

Classifying reasoning traces...


Classifying: 100%|██████████| 199/199 [36:19<00:00, 10.95s/it]

classification
hypothesis_testing       1911
evidence_gathering        846
hypothesis_generation     645
process_control           291
diagnosis                 231
Name: count, dtype: int64


In [ ]:
sentence_df = pd.read_csv(f"{output_dir}/sentence_classifications.csv")
results_df = pd.read_csv(f"{output_dir}/reasoning_traces.csv")

In [ ]:
from rich.console import Console
from rich.text import Text
from rich.panel import Panel
from ipywidgets import interact, Dropdown
import pandas as pd

# Color mapping based on your image
COLOR_MAP = {
    'process_control': '#FFC5C5',      # Light coral/pink
    'evidence_gathering': '#FFE5C5',    # Light peach
    'hypothesis_generation': '#FFFFC5', # Light yellow
    'hypothesis_testing': '#E5FFC5',    # Light green
    'diagnosis': '#C5E5FF'              # Light blue
}
console = Console()
def display_classified_reasoning(
    pmcid: str,
    reasoning_trace: str,
    sentence_df: pd.DataFrame
):
    """
    Display a reasoning trace with color-coded sentence classifications.

    Args:
        pmcid: Case identifier
        reasoning_trace: The full reasoning text
        sentence_df: DataFrame with pmcid, sentence, classification columns
    """

    console.clear()  # Add this line

    # Get classifications for this case
    case_sentences = sentence_df[sentence_df['pmcid'] == pmcid]

    if len(case_sentences) == 0:
        console.print(f"[red]No classifications found for PMCID: {pmcid}[/red]")
        return

    # Create colored text
    colored_text = Text()

    for _, row in case_sentences.iterrows():
        sentence = row['sentence']
        classification = row['classification']
        color = COLOR_MAP.get(classification, 'white')

        # Add sentence with background color and spacing
        colored_text.append(sentence, style=f"black on {color}")
        colored_text.append(" ")  # Space between sentences

    # Display in a panel
    console.print(Panel(
        colored_text,
        title=f"Case: {pmcid}",
        border_style="blue",
        expand=False
    ))

    # Display legend
    console.print("\n[bold]Legend:[/bold]")
    for classification, color in COLOR_MAP.items():
        legend_text = Text(f"  {classification.replace('_', ' ').title()}", style=f"black on {color}")
        console.print(legend_text)

def create_case_viewer(results_df: pd.DataFrame, sentence_df: pd.DataFrame):
    """
    Create an interactive viewer for cases with classified reasoning.

    Args:
        results_df: DataFrame with pmcid, reasoning_trace columns
        sentence_df: DataFrame with pmcid, sentence, classification columns
    """
    # Get list of PMCIDs that have classifications
    available_pmcids = sentence_df['pmcid'].unique().tolist()

    # Filter results_df to only include cases with classifications
    viewable_cases = results_df[results_df['pmcid'].isin(available_pmcids)]

    # Create dropdown options (index: pmcid)
    dropdown_options = {
        f"{idx}: {pmcid}": pmcid
        for idx, pmcid in enumerate(viewable_cases['pmcid'].unique())
    }

    def view_case(case_selection):
        """Display the selected case"""
        pmcid = case_selection

        # Get the reasoning trace
        case_row = results_df[results_df['pmcid'] == pmcid].iloc[0]
        reasoning_trace = case_row['reasoning_trace']

        # Display with colors
        display_classified_reasoning(pmcid, reasoning_trace, sentence_df)

    # Create interactive widget
    interact(
        view_case,
        case_selection=Dropdown(
            options=dropdown_options,
            description='Select Case:',
            style={'description_width': 'initial'}
        )
    )

# Usage:
# After running your classification pipeline, simply call:
create_case_viewer(results_df, sentence_df)

interactive(children=(Dropdown(description='Select Case:', options={'0: PMC9366264': 'PMC9366264', '1: PMC8938…

In [ ]:
def create_contrastive_agent(model) -> Agent:
    """
    Create an agent specifically for generating contrastive negative examples
    for steering vector training.

    Args:
        model: Configured model for generation
    """
    instructions = """
    You are an expert at generating contrastive examples for training steering vectors
    in medical reasoning models.

    Your task: Given a medical case and a sentence showing GOOD hypothesis generation,
    generate a CONTRASTIVE NEGATIVE example that shows the opposite behavior.

    GOOD hypothesis generation (positive examples) shows:
    - Considering multiple differential diagnoses
    - Keeping an open mind to various possibilities
    - Generating alternative explanations
    - Reconsidering initial impressions
    - Using phrases like "could be", "alternatively", "let me consider", "what about"

    BAD reasoning (negative examples) shows:
    - Anchoring on a single diagnosis prematurely
    - Jumping to conclusions without considering alternatives
    - Using definitive language too early: "this is clearly", "obviously", "definitely"
    - Pattern matching without differential reasoning
    - Closing off consideration of other diagnoses

    CRITICAL: Your generated negative sentence should be:
    1. Plausible - it should sound like something a model might actually say
    2. Contextually relevant - it should relate to the specific case details
    3. Similar length to the positive example
    4. A complete thought (not a fragment)
    5. Clearly demonstrating anchoring/hasty reasoning behavior

    OUTPUT FORMAT: Return ONLY the negative sentence, nothing else. No explanations,
    no preamble, just the contrastive sentence.

    EXAMPLE:
    Positive: "This could be benign metastasizing leiomyoma, but I should also consider leiomyosarcoma and metastatic disease."
    Negative: "The pulmonary nodules are clearly metastases from leiomyosarcoma."
    """

    agent = Agent(
        model,
        output_type=str,
        instructions=instructions,
    )
    return agent


async def generate_contrastive_negative(
    agent: Agent,
    case_prompt: str,
    positive_sentence: str,
    pmcid: str
) -> dict:
    """
    Generate a contrastive negative example for a hypothesis generation sentence.

    Args:
        agent: Agent to generate contrastive examples
        case_prompt: The medical case presentation
        positive_sentence: The hypothesis generation sentence (positive example)
        pmcid: Case identifier

    Returns:
        Dictionary with generated negative sentence
    """
    prompt = f"""Given a medical case and a sentence showing good hypothesis generation behaviour,
generate a contrastive negative example exhibits the exact opposite of this behaviour.

The positive example demonstrates:
- Considering multiple differential diagnoses
- Proposing new differentials
- Keeping an open mind
- Generating alternative possibilities

The negative example should demonstrate:
- Refusing or delaying e.g. I'll wait for more test results before considering anything. I can't speculate without more data.
- Anchoring on a single diagnosis prematurely e.g. The sunburst pattern is pathognomonic of osteosarcoma.
- Jumping to conclusions
- Closing off consideration of alternatives e.g. That's definitely not a parotid issue.

CASE:
{case_prompt}

POSITIVE EXAMPLE (good hypothesis generation):
{positive_sentence}

Generate a contrastive NEGATIVE example that shows hasty, anchored reasoning instead.
Output only the negative sentence, nothing else."""

    try:
        result = await agent.run(prompt)
        response = result.all_messages()[-1].text if hasattr(result.all_messages()[-1], 'text') else str(result.output)
        return {
            'pmcid': pmcid,
            'positive_sentence': positive_sentence,
            'negative_sentence': response.strip(),
            'success': True,
            'error': None
        }
    except Exception as e:
        print(e)
        return {
            'pmcid': pmcid,
            'positive_sentence': positive_sentence,
            'negative_sentence': None,
            'success': False,
            'error': str(e)
        }


In [ ]:



async def extract_contrastive_pairs(
    sentence_df: pd.DataFrame,
    case_prompts_dict: dict,
    contrastive_agent: Agent = None,
    max_concurrent: int = 10
) -> List[tuple]:
    """
    Extract contrastive pairs for steering vector training.

    Positive examples: hypothesis_generation sentences
    Negative examples: diagnosis sentences (from same pmcid)

    Args:
        sentence_df: DataFrame with pmcid, sentence, classification columns
        case_prompts_dict: Dict mapping pmcid -> case_prompt

    Returns:
        List of (pmcid, case_prompt, positive_suffix, negative_suffix) tuples
    """
    contrastive_pairs = []
    if contrastive_agent is None:
            raise ValueError("contrastive_agent must be provided when using 'generated' strategy")

    # Collect all hypothesis generation sentences
    hypothesis_cases = []
    for pmcid in sentence_df['pmcid'].unique():
        pmcid_df = sentence_df[sentence_df['pmcid'] == pmcid]
        hypothesis_sentences = pmcid_df[
            pmcid_df['classification'] == 'hypothesis_generation'
        ]['sentence'].tolist()

        case_prompt = case_prompts_dict.get(pmcid, "")
        if not case_prompt:
            continue

        for hyp_sentence in hypothesis_sentences:
            hypothesis_cases.append({
                'pmcid': pmcid,
                'case_prompt': case_prompt,
                'positive_sentence': hyp_sentence
            })

    # Generate negatives with concurrency control
    semaphore = asyncio.Semaphore(max_concurrent)

    async def generate_with_semaphore(case):
        async with semaphore:
            return await generate_contrastive_negative(
                contrastive_agent,
                case['case_prompt'],
                case['positive_sentence'],
                case['pmcid']
            )

    tasks = [generate_with_semaphore(case) for case in hypothesis_cases]

    results = []
    for task in tqdm(asyncio.as_completed(tasks), total=len(tasks), desc="Generating negatives"):
        result = await task
        results.append(result)

    # Convert to contrastive pairs
    for result in results:
        if result['success'] and result['negative_sentence']:
            # Find the matching case
            matching_case = next(
                (c for c in hypothesis_cases if c['pmcid'] == result['pmcid']
                  and c['positive_sentence'] == result['positive_sentence']),
                None
            )
            if matching_case:
                contrastive_pairs.append({
                    'pmcid': result['pmcid'],
                    'case_prompt': matching_case['case_prompt'],
                    'positive_suffix': result['positive_sentence'],
                    'negative_suffix': result['negative_sentence']
                })

    return contrastive_pairs

def create_steering_dataset(
    contrastive_pairs: List[dict],
    tokenizer,
    user_tag: str,
    asst_tag: str,
    max_suffix_tokens: int = 30
) -> List:
    """
    Create dataset for steering vector training.

    Args:
        contrastive_pairs: List of dicts with case_prompt, positive_suffix, negative_suffix
        tokenizer: Tokenizer for encoding
        user_tag: User tag for prompt template
        asst_tag: Assistant tag for prompt template
        max_suffix_tokens: Maximum tokens in suffix

    Returns:
        List of DatasetEntry objects
    """
    from dataclasses import dataclass

    @dataclass
    class DatasetEntry:
        positive: str
        negative: str

    dataset = []

    for pair in tqdm(contrastive_pairs, desc="Creating steering dataset"):
        case_prompt = pair['case_prompt']

        # Encode and truncate positive suffix
        pos_tokens = tokenizer.encode(pair['positive_suffix'], add_special_tokens=False)
        pos_tokens = pos_tokens[:max_suffix_tokens]

        # Encode and truncate negative suffix
        neg_tokens = tokenizer.encode(pair['negative_suffix'], add_special_tokens=False)
        neg_tokens = neg_tokens[:max_suffix_tokens]

        # Create multiple truncations for each suffix (like the paper)
        if len(pos_tokens) > 5 and len(neg_tokens) > 5:
            # Use same truncation points for both to keep them comparable
            max_truncations = min(len(pos_tokens) - 5, len(neg_tokens) - 5)

            for i in range(1, max_truncations):
                pos_suffix = tokenizer.decode(pos_tokens[:i], skip_special_tokens=True)
                neg_suffix = tokenizer.decode(neg_tokens[:i], skip_special_tokens=True)

                dataset.append(
                    DatasetEntry(
                        positive=f"{user_tag} {case_prompt} {asst_tag} {pos_suffix}",
                        negative=f"{user_tag} {case_prompt} {asst_tag} {neg_suffix}"
                    )
                )

    return dataset

In [ ]:
contrastive_agent = create_contrastive_agent(verification_model)

case_prompts_dict = dict(zip(results_df['pmcid'], results_df['case_prompt']))
contrastive_pairs = await extract_contrastive_pairs(sentence_df, case_prompts_dict, contrastive_agent = contrastive_agent, max_concurrent=max_concurrent)
pairs_df = pd.DataFrame(contrastive_pairs)
pairs_df.to_csv(f"{output_dir}/contrastive_pairs.csv", index=False)
print(f"Extracted {len(contrastive_pairs)} contrastive pairs")

Extracted 645 contrastive pairs


In [ ]:
pairs_df

,pmcid,case_prompt,positive_suffix,negative_suffix
0,PMC3970369,A 51-year-old Caucasian man was admitted after...,Other differentials: maybe a central cause.,This is obviously an ischemic stroke given the...
1,PMC3970369,A 51-year-old Caucasian man was admitted after...,Bilateral thalamic lesions—what could cause that?,These bilateral thalamic lesions are obviously...
2,PMC3970369,A 51-year-old Caucasian man was admitted after...,Stroke?,This is clearly an acute ischemic stroke in th...
3,PMC3970369,A 51-year-old Caucasian man was admitted after...,"Thalamic infarction can occur, but if the angi...",Given the bilateral thalamic changes and norma...
4,PMC3970369,A 51-year-old Caucasian man was admitted after...,Or a non-vascular issue.,This is clearly vascular - there's no point co...
...,...,...,...,...
640,PMC8943113,A 70-year-old woman was evaluated for an incid...,Or maybe myeloma?,Given her age and the pancreatic cancer histor...
641,PMC8943113,A 70-year-old woman was evaluated for an incid...,Maybe the adrenal was involved in the primary ...,The adrenal involvement definitely proves this...
642,PMC8943113,A 70-year-old woman was evaluated for an incid...,Another angle: vertebral destruction can be fr...,Based on the CT findings of vertebral destruct...
643,PMC8943113,A 70-year-old woman was evaluated for an incid...,"Alternatively, maybe another pathogen?",This is definitely just a Candida infection - ...


# Steering Experiments

In [ ]:
import json

import torch
from transformers import AutoModelForCausalLM, AutoTokenizer

from repeng import ControlVector, ControlModel, DatasetEntry

model_name = "mistralai/Mistral-7B-Instruct-v0.1"

tokenizer = AutoTokenizer.from_pretrained(model_name)
tokenizer.pad_token_id = 0

model = AutoModelForCausalLM.from_pretrained(model_name, torch_dtype=torch.float16)
model = model.to("cuda:0" if torch.cuda.is_available() else "cpu")
model = ControlModel(model, list(range(-5, -18, -1)))

user_tag, asst_tag = "[INST]", "[/INST]"

/usr/lib/python3.12/ast.py:52: RuntimeWarning: coroutine 'extract_contrastive_pairs' was never awaited
  return compile(source, filename, mode, flags,


tokenizer_config.json: 0.00B [00:00, ?B/s]

tokenizer.model:   0%|          | 0.00/493k [00:00<?, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/414 [00:00<?, ?B/s]

...

Fetching 2 files:   0%|          | 0/2 [00:00<?, ?it/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/4.54G [00:00<?, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/9.94G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

In [ ]:
# If you have contrastive_pairs as a list of dicts:
subset_pmcids = set(medcase_df[1:200]["pmcid"].tolist())

filtered_contrastive_pairs = [
    pair for pair in contrastive_pairs
    if pair['pmcid'] in subset_pmcids
]

In [ ]:
filtered_contrastive_pairs

[{'pmcid': 'PMC3970369',
  'case_prompt': 'A 51-year-old Caucasian man was admitted after home nursing services noted sudden onset dysarthria and unilateral facial nerve paresis. His history included hypothyroidism, peptic ulcer disease, and prior alcohol-related polyneuropathy. He was brought by air ambulance as a possible thrombolysis candidate. \n\nOn arrival he was fully conscious but perplexed and disoriented to date and time. Examination showed equal, reactive pupils; intact corneal reflexes and visual fields; and dysconjugate midline gaze with bilateral abduction weakness causing diplopia on lateral gaze. There was no nystagmus, facial asymmetry, or tongue deviation. Strength testing revealed generalized weakness without focal paresis. Deep tendon reflexes were 2+ with flexor plantar responses, and sensation was intact bilaterally. \n\nBlood pressure was 170/110 mmHg. ECG showed normal sinus rhythm with T-wave inversion in inferior and lateral leads. Urgent unenhanced head CT de

In [ ]:
steering_dataset = create_steering_dataset(
        filtered_contrastive_pairs,
        tokenizer,
        user_tag,
        asst_tag,
        max_suffix_tokens=80
    )
print(f"Created {len(steering_dataset)} dataset entries")

Creating steering dataset: 100%|██████████| 645/645 [00:00<00:00, 1880.80it/s]

Created 7570 dataset entries


In [ ]:
def template(persona: str, case_prompt: str, suffix: str) -> str:
    return f"{user_tag} You are a doctor that is {persona}. {case_prompt} {asst_tag} {suffix}"

dataset = []
for idx, row in tqdm(train.iterrows(), total=len(train), desc="Cases"):
    # Get the case prompt
    case_tokens = tokenizer.tokenize(row['case_prompt'])
    case_truncated = tokenizer.convert_tokens_to_string(case_tokens)

    # Get the diagnostic reasoning as suffix
    reasoning = row['diagnostic_reasoning']
    reasoning_tokens = tokenizer.tokenize(reasoning)
    reasoning_tokens = reasoning_tokens[:30]
    # Create multiple truncations of the reasoning (like the paper does)
    # Always chop off last 5 tokens so model has something to complete
    for i in range(1, len(reasoning_tokens) - 5):
        truncated_reasoning = tokenizer.convert_tokens_to_string(reasoning_tokens[:i])

        dataset.append(
            DatasetEntry(
                positive=template(
                    "systematic, starting with broad differentials with no anchoring bias",
                    case_truncated,
                    truncated_reasoning
                ),
                negative=template(
                    "hasty, impulsive, and says the first knee-jerk response",
                    case_truncated,
                    truncated_reasoning
                )
            )
        )




Cases: 100%|██████████| 49/49 [00:00<00:00, 375.77it/s]


In [ ]:
# print some example entries
for i in range(3):
    print(f"dataset[{i}].positive:", dataset[i].positive)
    print(f"dataset[{i}].negative:", dataset[i].negative)

dataset[0].positive: [INST] You are a doctor that is systematic, starting with broad differentials with no anchoring bias. A 27-year-old left-hand–dominant man presented with bilateral clicking and discomfort of several years’ duration, localized to the anterior aspects of both elbows and slowly worsening despite simple analgesia and physiotherapy. He reported intermittent paraesthesia in the forearms, particularly after physical activity. Symptoms were exacerbated by pull-ups and heavy lifting at work. He was otherwise healthy. On examination, both elbows had a full range of motion, with no instability or weakness. Grip and grind tests were negative bilaterally. There was no objective sensory disturbance, and Tinel’s sign was negative. The hook test for distal biceps rupture was negative, and provocative testing of the biceps produced no pain. While moving the left elbow, a palpable click over the lateral biceps tendon reduced with digital pressure. Plain radiographs and MRI of both e

In [ ]:
model.reset() # make sure you always reset the model before training a new vector
control_vector = ControlVector.train(
    model,
    tokenizer,
    steering_dataset
)

100%|██████████| 31/31 [00:18<00:00,  1.64it/s]


In [ ]:
# If you have contrastive_pairs as a list of dicts:
subset_pmcids = set(medcase_df[1:200]["pmcid"].tolist())

filtered_contrastive_pairs = [
    pair for pair in contrastive_pairs
    if pair['pmcid'] in subset_pmcids
]

print(f"Original pairs: {len(contrastive_pairs)}")
print(f"Filtered pairs: {len(filtered_contrastive_pairs)}")

Original pairs: 645
Filtered pairs: 645


In [ ]:
medcase_df.iloc[1001]

,1001
Unnamed: 0,1118
pmcid,PMC10868934
title,Cystic brain metastases radiologically simulat...
journal,São Paulo Medical Journal
article_link,https://www.ncbi.nlm.nih.gov/pmc/articles/PMC1...
publication_date,2011-09-01
text,INTRODUCTION Brain metastases are common compl...
case_prompt,A 45-year-old woman with a history of breast c...
diagnostic_reasoning,1. Neurocysticercosis considered because of th...
final_diagnosis,Metastatic breast cancer


In [ ]:
from IPython.display import display, HTML

def show_case(idx):
    row = medcase_df.iloc[idx]
    html = f"""
    <div style='border: 2px solid #4CAF50; padding: 15px; margin: 10px; border-radius: 5px;'>
        <h3 style='color: #4CAF50;'>Case {idx}: {row['pmcid']}</h3>
        <p><strong>True Diagnosis:</strong> <span style='color: green;'>{row['final_diagnosis']}</span></p>
        <hr>
        <h4>Case Prompt:</h4>
        <div style='padding: 10px; border-radius: 5px; max-height: 400px; overflow-y: auto;'>
            {row['case_prompt'].replace('\n', '<br>')}
        </div>
        <h4>Dataset Reasoning Trace:</h4>
        <div style='padding: 10px; border-radius: 5px; max-height: 400px; overflow-y: auto;'>
            {row['diagnostic_reasoning'].replace('\n', '<br>')}
        </div>
    </div>
    """
    display(HTML(html))

# View incorrect cases
show_case(1002)

In [ ]:
test_prompt = medcase_df.iloc[500]["case_prompt"]

In [ ]:
# this question is taken from the paper
input = f"{user_tag} You're a doctor. Think through the following medical case. {test_prompt} {asst_tag}"

# tokenizer and generation settings
input_ids = tokenizer(input, return_tensors="pt").to(model.device)
settings = {
    "pad_token_id": tokenizer.eos_token_id, # silence warning
    "do_sample": False, # temperature=0
    "max_new_tokens": 512,
    "repetition_penalty": 1.1, # reduce control jank
}

print("==baseline")
model.reset()
print(tokenizer.decode(model.generate(**input_ids, **settings).squeeze()))

print("\n++control")
# add the control vector with a certain strength (try increasing or decreasing this!)
model.set_control(control_vector, 1.0)
print(tokenizer.decode(model.generate(**input_ids, **settings).squeeze()))

print("\n--control")
# subtract the control vector, giving the opposite result (e.g. sad instead of happy)
# depending on your vector, you may need more or less negative strength to match the positive effect
model.set_control(control_vector, -1.0)
print(tokenizer.decode(model.generate(**input_ids, **settings).squeeze()))
model.reset()

==baseline
<s> [INST] You're a doctor. Think through the following medical case. A 34-year-old man from Punjab, India, presented with a 7-year history of gradual bilateral intention tremor initially diagnosed as spinocerebellar ataxia. Over the past 2 years, he developed progressive psychiatric symptoms including severe depression, anxiety, and mood lability managed with mood stabilizers, antidepressants, and antipsychotics with limited benefit. After emigrating to Canada, he experienced rapid functional decline over one year, with gait instability, rigidity, scanning speech, dysphagia, choking episodes, restlessness, sexual disinhibition, aggression, and social inappropriateness. By summer 2011, he was nonambulatory and had an acute in-hospital episode of dystonia, prompting admission. 

Family history was notable only for the patient’s maternal grandfather, who developed rapidly progressive gait disturbance and cognitive impairment at age 62 and died ~8 months later; there was no kno

In [ ]:
test_cases

,Unnamed: 0,pmcid,title,journal,article_link,publication_date,text,case_prompt,diagnostic_reasoning,final_diagnosis
1001,1118,PMC10868934,Cystic brain metastases radiologically simulat...,São Paulo Medical Journal,https://www.ncbi.nlm.nih.gov/pmc/articles/PMC1...,2011-09-01,INTRODUCTION Brain metastases are common compl...,A 45-year-old woman with a history of breast c...,1. Neurocysticercosis considered because of th...,Metastatic breast cancer
1002,1119,PMC8927634,Calcifying Aponeurotic Fibroma: A Multimodal I...,Radiology Case Reports,https://www.ncbi.nlm.nih.gov/pmc/articles/PMC8...,2022-03-02,"Introduction In 1953, Keasbey first described ...",A 64-year-old woman presented with a 3-year hi...,1. Synovial osteochondromatosis was initially ...,Calcifying aponeurotic fibroma
1003,1120,PMC9019870,Aggressive fibromatosis in the submandibular r...,Clinical Case Reports,https://www.ncbi.nlm.nih.gov/pmc/articles/PMC9...,2022-04-20,1 INTRODUCTION Fibrous tumors and tumor‐like c...,A 5-year-old girl presented with a 2-month his...,1. Inflammatory lesion — “The provisional clin...,Aggressive fibromatosis
1004,1121,PMC4073195,Mesothelial cyst with endometriosis mimicking ...,Journal of Surgical Case Reports,https://www.ncbi.nlm.nih.gov/pmc/articles/PMC4...,2014-06-27,INTRODUCTION Extragenital endometriosis in the...,A 35-year-old woman was referred for evaluatio...,1. Nuck cyst — “We diagnosed the right inguina...,Mesothelial cyst
1005,1122,PMC9829880,Splenic lymphangioma in adult patient treated ...,SAGE Open Medical Case Reports,https://www.ncbi.nlm.nih.gov/pmc/articles/PMC9...,2023-01-03,Introduction Lymphangiomas are considered to b...,A 55-year-old woman presented with a 3-day his...,1. Hamartoma was considered as part of the ini...,splenic lymphangioma
...,...,...,...,...,...,...,...,...,...,...
1095,1228,PMC6159200,Gastric and colonic erosion caused by laparosc...,BJR Case Reports,https://www.ncbi.nlm.nih.gov/pmc/articles/PMC6...,2017-03-10,Introduction According to the Centers for Dise...,A 62-year-old woman with a history of lupus an...,1. Diverticulitis considered because of LLQ pa...,gastric and colonic erosion
1096,1229,PMC11964546,Epidermoid cyst of the penis: A case report,International Journal of Surgery Case Reports,https://www.ncbi.nlm.nih.gov/pmc/articles/PMC1...,2025-03-18,1 Introduction Epidermoid cysts (EC) are slow-...,A 35-year-old man with no significant medical ...,1. Infectious etiologies were considered but r...,Epidermoid cyst
1097,1230,PMC11899630,Pelvic Actinomycosis and Diagnostic Complexity...,Healthcare,https://www.ncbi.nlm.nih.gov/pmc/articles/PMC1...,2025-02-24,1. Introduction Actinomyces are obligate anaer...,A 48-year-old woman presented to the emergency...,1. Ovarian malignancy — “Based on ultrasound e...,Actinomycosis
1098,1231,PMC6509900,Management of subcutaneous facial emphysema se...,Clinical Case Reports,https://www.ncbi.nlm.nih.gov/pmc/articles/PMC6...,2019-04-16,1 INTRODUCTION Subcutaneous facial emphysema (...,A 43-year-old man was referred to the emergenc...,1. A hypersensitivity reaction was considered ...,cervicofacial emphysema


In [ ]:
# Set test cases
test_cases = medcase_df[201:300]

# Initialize results list
results = []



# Loop through all test cases
for idx, row in tqdm(test_cases.iterrows(), total=len(test_cases), desc="Processing cases"):
    case_prompt = row['case_prompt']
    true_diagnosis = medcase_df.loc[idx, 'final_diagnosis']
    diagnostic_reasoning = medcase_df.loc[idx, 'diagnostic_reasoning']
    pmcid = medcase_df.loc[idx, 'pmcid']

    prompt = f"""{user_tag}
    Read the following case presentation and give the most likely diagnosis.
    You must think very carefully, considering the information available to you. Write out your full reasoning.
    Then, output the final diagnosis (just the name of the disease/entity) within the tags <answer> ... </answer>.
    Your final answer can not be ambiguous even if you think you don't have enough information.
    ---------------------------------------
    CASE PRESENTATION
    ---------------------------------------
    {case_prompt}
    ---------------------------------------
    OUTPUT TEMPLATE
    ---------------------------------------
    <answer>
    ...the name of the disease/entity...
    </answer> {asst_tag}"""

    # Test with three steering conditions
    for steer_type, steer_coef in [("baseline", 0), ("positive", 0.5), ("negative", -0.5)]:
        # Create input prompt
        input_text = prompt

        # Tokenize
        input_ids = tokenizer(input_text, return_tensors="pt").to(model.device)
        settings = {
            "pad_token_id": tokenizer.eos_token_id,
            "do_sample": False,
            "max_new_tokens": 512,
            "repetition_penalty": 1.1,
        }

        # Generate with appropriate steering
        model.reset()
        if steer_coef != 0:
            model.set_control(control_vector, steer_coef)

        output = model.generate(**input_ids, **settings)
        raw_response = tokenizer.decode(output.squeeze(), skip_special_tokens=True)
        print(raw_response)
        # Extract predicted diagnosis
        after_inst = raw_response.split('[/INST]', 1)[1]
        answer_match = re.search(r'<answer>(.*?)</answer>', after_inst, re.DOTALL | re.IGNORECASE)
        predicted_diagnosis = answer_match.group(1).strip() if answer_match else raw_response
        print(predicted_diagnosis)
        print(true_diagnosis)
        # Verify correctness using verification agent
        verification_result = await verify_diagnosis(
            verification_agent,
            predicted_diagnosis if predicted_diagnosis else "None",
            true_diagnosis
        )

        is_correct = verification_result['is_correct']
        verification_explanation = verification_result['explanation']

        # Append results
        results.append({
            'pmcid': pmcid,
            'case_prompt': case_prompt,
            'diagnostic_reasoning': diagnostic_reasoning,
            'true_diagnosis': true_diagnosis,
            'steer_type': steer_type,
            'raw_response': raw_response,
            'predicted_diagnosis': predicted_diagnosis,
            'is_correct': is_correct,
            'verification_explanation': verification_explanation
        })

        model.reset()

# Create results dataframe
results_df = pd.DataFrame(results)

Processing cases:   0%|          | 0/99 [00:00<?, ?it/s]

[INST]
    Read the following case presentation and give the most likely diagnosis.
    You must think very carefully, considering the information available to you. Write out your full reasoning.
    Then, output the final diagnosis (just the name of the disease/entity) within the tags <answer> ... </answer>.
    Your final answer can not be ambiguous even if you think you don't have enough information.
    ---------------------------------------
    CASE PRESENTATION
    ---------------------------------------
    A 55-year-old man presented with a 6-year history of dysesthesia described as sensations of movement beneath the skin, accompanied by pruritus, erythema, burning, and pain. He believed “flies and worms” were emerging from his skin and attributed his symptoms to a parasitic infection, noting prior camping and travel—including South Carolina, South Florida, the Bahamas, Kansas, Louisiana, Texas, New York, Colorado, and Nevada—in which he felt he might have been exposed. His me

Processing cases:   1%|          | 1/99 [00:28<45:59, 28.16s/it]

[INST]
    Read the following case presentation and give the most likely diagnosis.
    You must think very carefully, considering the information available to you. Write out your full reasoning.
    Then, output the final diagnosis (just the name of the disease/entity) within the tags <answer> ... </answer>.
    Your final answer can not be ambiguous even if you think you don't have enough information.
    ---------------------------------------
    CASE PRESENTATION
    ---------------------------------------
    A 6-year-old boy from a rural area presented with a one-month history of a progressively enlarging right axillary mass. He denied pain or inflammatory signs but reported limited abduction of the right upper limb. He was initially diagnosed with pyogenic adenitis and treated with cephalexin without improvement. He also described transient enlargements of cervical and inguinal lymph nodes and intermittent arthralgias of the wrists, ankles, and knees, all resolving spontaneousl

Processing cases:   2%|▏         | 2/99 [00:50<39:46, 24.60s/it]

...

Processing cases:  98%|█████████▊| 97/99 [36:29<00:41, 20.69s/it]

[INST]
    Read the following case presentation and give the most likely diagnosis.
    You must think very carefully, considering the information available to you. Write out your full reasoning.
    Then, output the final diagnosis (just the name of the disease/entity) within the tags <answer> ... </answer>.
    Your final answer can not be ambiguous even if you think you don't have enough information.
    ---------------------------------------
    CASE PRESENTATION
    ---------------------------------------
    An eight-year-old female spayed Great Dane was presented to the emergency department of a referral hospital after radiographs confirmed gastric dilatation–volvulus. The dog was weak and unwilling to walk. Emergency surgery with an incisional gastropexy was performed. For the procedure, the dog was placed in standard dorsal recumbency and secured to the operating table with four limb ties. Postoperatively, the dog was unwilling to rise and could not be walked frequently overn

Processing cases:  99%|█████████▉| 98/99 [36:42<00:18, 18.09s/it]

[INST]
    Read the following case presentation and give the most likely diagnosis.
    You must think very carefully, considering the information available to you. Write out your full reasoning.
    Then, output the final diagnosis (just the name of the disease/entity) within the tags <answer> ... </answer>.
    Your final answer can not be ambiguous even if you think you don't have enough information.
    ---------------------------------------
    CASE PRESENTATION
    ---------------------------------------
    A 55-year-old Japanese man with a 13-year history of diabetes was brought to the hospital after a sudden loss of consciousness and bilateral convulsions (day 0). On examination, he was afebrile (temperature 36.8 °C), hypertensive (blood pressure 142/92 mmHg), and tachycardic (heart rate 108 beats/min). He had left-sided hemiparesis that was unresponsive to painful stimuli. Laboratory evaluation showed a pH of 6.95 and a blood glucose level of 486 mg/dL, consistent with diabe

Processing cases: 100%|██████████| 99/99 [36:58<00:00, 22.41s/it]


In [ ]:

# Display function
from IPython.display import display, HTML

def show_case(idx):
    row = results_df.iloc[idx]
    html = f"""
    <div style='border: 2px solid #4CAF50; padding: 15px; margin: 10px; border-radius: 5px;'>
        <h3 style='color: #4CAF50;'>Case {idx}: {row['pmcid']} - Steering: {row['steer_type']}</h3>
        <p><strong>True Diagnosis:</strong> <span style='color: green;'>{row['true_diagnosis']}</span></p>
        <p><strong>Predicted:</strong> <span style='color: {"green" if row["is_correct"] == "y" else "red"};'>{row['predicted_diagnosis']}</span></p>
        <p><strong>Correct:</strong> {row['is_correct']}</p>
        <hr>
        <h4>Case Prompt:</h4>
        <div style='padding: 10px; border-radius: 5px; max-height: 400px; overflow-y: auto;'>
            {row['case_prompt'].replace('\n', '<br>')}
        </div>
        <h4>Model Raw Response:</h4>
        <div style='padding: 10px; border-radius: 5px; max-height: 400px; overflow-y: auto;'>
            {row['raw_response'].replace('\n', '<br>')}
        </div>
        <h4>Dataset Reasoning Trace:</h4>
        <div style='padding: 10px; border-radius: 5px; max-height: 400px; overflow-y: auto;'>
            {row['diagnostic_reasoning'].replace('\n', '<br>')}
        </div>
        <h4>Verification Explanation:</h4>
        <div style='padding: 10px; border-radius: 5px; max-height: 400px; overflow-y: auto;'>
            {row['verification_explanation'].replace('\n', '<br>')}
        </div>
    </div>
    """
    display(HTML(html))

# Interactive widget
from ipywidgets import interact, Dropdown

idx = results_df.index.tolist()

@interact(case_idx=Dropdown(options=idx, description='Case:'))
def explore_cases(case_idx):
    show_case(case_idx)

interactive(children=(Dropdown(description='Case:', options=(0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14,…

In [ ]:
# Calculate win rates
print("\n" + "="*50)
print("DIAGNOSTIC ACCURACY BY STEERING TYPE")
print("="*50)

# Group by steering type and calculate accuracy
accuracy_by_steer = results_df.groupby('steer_type').agg({
    'is_correct': lambda x: (x == 'y').sum() / len(x) * 100
}).round(2)

print(accuracy_by_steer)

# More detailed breakdown
for steer_type in ['baseline', 'positive', 'negative']:
    steer_results = results_df[results_df['steer_type'] == steer_type]
    total = len(steer_results)
    correct = (steer_results['is_correct'] == 'y').sum()
    accuracy = (correct / total * 100) if total > 0 else 0

    print(f"\n{steer_type.upper()}:")
    print(f"  Correct: {correct}/{total} ({accuracy:.2f}%)")

# Calculate which steering wins
print("\n" + "="*50)
print("STEERING COMPARISON")
print("="*50)

pivot_df = results_df.pivot_table(
    index='pmcid',
    columns='steer_type',
    values='is_correct',
    aggfunc='first'
)

# Count wins
baseline_correct = (pivot_df['baseline'] == 'y').sum()
positive_correct = (pivot_df['positive'] == 'y').sum()
negative_correct = (pivot_df['negative'] == 'y').sum()

print(f"Baseline correct: {baseline_correct}")
print(f"Positive steering correct: {positive_correct}")
print(f"Negative steering correct: {negative_correct}")

# Cases where positive beats baseline
positive_wins = ((pivot_df['positive'] == 'y') & (pivot_df['baseline'] == 'n')).sum()
baseline_wins = ((pivot_df['baseline'] == 'y') & (pivot_df['positive'] == 'n')).sum()

print(f"\nPositive steering beats baseline: {positive_wins} cases")
print(f"Baseline beats positive steering: {baseline_wins} cases")


DIAGNOSTIC ACCURACY BY STEERING TYPE
            is_correct
steer_type            
baseline         18.18
negative         12.12
positive         14.14

BASELINE:
  Correct: 18/99 (18.18%)

POSITIVE:
  Correct: 14/99 (14.14%)

NEGATIVE:
  Correct: 12/99 (12.12%)

STEERING COMPARISON
Baseline correct: 18
Positive steering correct: 14
Negative steering correct: 12

Positive steering beats baseline: 3 cases
Baseline beats positive steering: 7 cases
